In [1]:
import pandas as pd
import numpy as np
import re, os
from pathlib import Path
from tqdm import tqdm
import pyarrow as pa
import pyarrow.parquet as pq
from pathlib import Path

In bash run ```$ cat *.txt>>combined.txt``` to concatenate all movie rating files into one

In [2]:
rating_file = Path('../data/raw/combined.txt')

In [4]:
with open(rating_file,'r',encoding='utf-8',errors='ignore') as f:
    data = f.readlines()

In [5]:
len(data)

100498277

In [6]:
rating_data=[]
for i,d in tqdm(enumerate(data),total=len(data)):
    d=d.strip()
    f=re.search('\d+:',d)
    if f is not None:
        movie = d[:-1]
        continue
    rating=d.split(',')
    rating_data.append((movie,*rating))
rating_df = pd.DataFrame(rating_data,columns=['movie_id','user_id','rating','date'])

100%|██████████| 100498277/100498277 [04:20<00:00, 385081.62it/s]


In [7]:
rating_df.head()

,movie_id,user_id,rating,date
0,1,1488844,3,2005-09-06
1,1,822109,5,2005-05-13
2,1,885013,4,2005-10-19
3,1,30878,4,2005-12-26
4,1,823519,3,2004-05-03


In [9]:
movie_count = rating_df.groupby('movie_id')['rating'].count()

In [10]:
user_count = rating_df.groupby('user_id')['rating'].count()

In [18]:
user_count.sort_values(ascending=False)[:10000]

user_id
305344     17653
387418     17436
2439493    16565
1664010    15813
2118461    14831
           ...  
2304247     1102
2170242     1102
2337776     1102
2170243     1102
54333       1102
Name: rating, Length: 10000, dtype: int64

In [27]:
movie_count.sort_values(ascending=False)[:1000]

movie_id
5317     232944
15124    216596
14313    200832
15205    196397
1905     193941
          ...  
17424     26061
1682      25961
10080     25954
9433      25905
8818      25873
Name: rating, Length: 1000, dtype: int64

In [21]:
sl_movies = movie_count.sort_values(ascending=False)[:1000].index

In [26]:
rating_df[rating_df.movie_id.isin(sl_movies)].groupby('user_id')['rating'].count().sort_values(ascending=False)[:10000]

user_id
716173     998
387418     996
2439493    990
2606799    989
1314869    988
          ... 
1984604    569
1409759    569
1725635    569
1671431    569
2075525    569
Name: rating, Length: 10000, dtype: int64

In [30]:
sl_users = user_count.sort_values(ascending=False)[:10000].index

In [34]:
sl_movies = rating_df[rating_df.user_id.isin(sl_users)].groupby('movie_id')['rating'].count().sort_values(ascending=False)[:1000].index

In [35]:
df = rating_df[rating_df.user_id.isin(sl_users) & rating_df.movie_id.isin(sl_movies)]

In [36]:
df.head()

,movie_id,user_id,rating,date
52525,28,1990901,5,2004-02-16
52528,28,765331,3,2003-03-20
52529,28,1987434,4,2005-07-14
52532,28,2193455,4,2005-01-19
52533,28,1468812,4,2003-01-14


In [37]:
df.shape

(6491181, 4)

In [38]:
table = pa.Table.from_pandas(df)

In [39]:
pq.write_table(table, 'sl_rating.parquet')